토픽 모델링의 대표적인 알고리즘은 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)라 할 수 있다.


참고 링크 : https://lettier.com/projects/lda-topic-modeling/

# 1. 잠재 디리클레 할당(이하 LDA) 개요

LDA를 일종의 블랙 박스로 보고 LDA에 문서 집합을 입력하면,
어떤 결과를 보여주는지 아래와 같이 살펴보자. 

아래와 같은 3개의 문서가 존재한다면 직관적을 토픽 모델링을 할 수 있으나,
실제 수십만개 이상의 문서가 있는 경우 직접 토픽을 찾아내기가 어렵다

* 문서1 : 저는 사과랑 바나나를 먹어요
* 문서2 : 우리는 귀여운 강아지가 좋아요
* 문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요 

LDA를 수행할 때 문서 집합에서 토픽이 몇 개가 존재할지 가정하는 것은 사용자가 해야 할 일이다. 만약 위의 문서 3개 중에서 2개의 토픽을 찾으라고 한다면

즉, 토픽의 개수를 의미하는 변수를 k라고 하였을 때, k를 2로 한다는 의미이다.

#### 1. 실습을 통한 이해 -1) News_data_headline

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
chicken_review = pd.read_csv('D:/pythonTest/DataShipJo/chicken_train_최종.csv', names=['번호','가게명','총평점','주소','아이디','평점','날짜','리뷰'])
chicken_review.head()

,번호,가게명,총평점,주소,아이디,평점,날짜,리뷰
0,0,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,de**님,1,2시간 전,잘 먹었습니다
1,1,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,pr**님,5,4시간 전,양이 많이 줄어든 느낌 이네요.
2,2,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,sk**님,5,8시간 전,오늘도 맛나게 잘 먹었습니다~~~
3,3,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,gh**님,5,10시간 전,맛있어요 양도 많아요
4,4,BHC-시립대점,4.8,서울특별시 동대문구 전농동 295-9 성진빌딩 295-9 동광교회 1층,ki**님,5,17시간 전,맛있게 잘먹었어요 다음에 또주문할께요


In [3]:
print('치킨 리뷰 개수 :', len(chicken_review))

치킨 리뷰 개수 : 99494


In [4]:
chicken_review.drop(chicken_review[['번호','가게명','총평점','주소','아이디','평점','날짜']], axis=1, inplace=True)
chicken_review.head()

,리뷰
0,잘 먹었습니다
1,양이 많이 줄어든 느낌 이네요.
2,오늘도 맛나게 잘 먹었습니다~~~
3,맛있어요 양도 많아요
4,맛있게 잘먹었어요 다음에 또주문할께요


## 2) 리뷰 전처리

In [5]:
chicken_review.isna().sum()

리뷰    0
dtype: int64

In [6]:
# # 결측값 열 삭제
# chicken_review.dropna(inplace=True)

In [7]:
# # 결측값 대체
# chicken_review['리뷰'].fillna('꿀맛', inplace=True)

### 불용어 제거, 표제어 추출, 길이가 짧은 단어 제거라는 세 가지 전처리 기법을 사용합니다.

In [8]:
# NLTK의 word_tokenize를 통해 단어 토큰화를 수행한다
chicken_review['리뷰'] = chicken_review.apply(lambda row: nltk.word_tokenize(str(row['리뷰'])), axis=1)

In [9]:
chicken_review.head()

,리뷰
0,"[잘, 먹었습니다]"
1,"[양이, 많이, 줄어든, 느낌, 이네요, .]"
2,"[오늘도, 맛나게, 잘, 먹었습니다~~~]"
3,"[맛있어요, 양도, 많아요]"
4,"[맛있게, 잘먹었어요, 다음에, 또주문할께요]"


### 불용어 사전

In [10]:
# RANKS NL에 제공해주는 한국어 불용어 사전
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

# 사용자가 지정하는 불용어를 stopwords 안에 추가로 저장
chicken_review_stopwords = ['시립대', '이네요', '다음에',
                           '느낌', '오늘도', '먹었습니다', '부탁드렸는데제', '휴', '아이구', '아이쿠', '역시',
                            '역시', 'ㅎㅎ', '후참', '조금', '그래도', '진짜', '좋아요', '처음', '시켰는데',
                            '엄청', '아주', '근데', '항상', '먹고', '맛있게', '맛있어서', '먹었어요',
                            'ㅠㅠ', '정말', '너무', '언제나', '잘먹었습니다', '감사합니다', '맛있고', '같아요', '오랜만에', '그냥', '자주']
for word in chicken_review_stopwords:
    stopwords.append(word)

### 불용어 제거

In [11]:
chicken_review['리뷰'] = chicken_review['리뷰'].apply(lambda x: [word for word in x if word not in (stopwords)])

In [12]:
chicken_review.head()

,리뷰
0,[잘]
1,"[양이, 많이, 줄어든, .]"
2,"[맛나게, 잘, 먹었습니다~~~]"
3,"[맛있어요, 양도, 많아요]"
4,"[잘먹었어요, 또주문할께요]"


### 정규 표현식 처리와 명사 추출

In [13]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')         # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', str(text))             # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [14]:
chicken_review.head()

,리뷰
0,[잘]
1,"[양이, 많이, 줄어든, .]"
2,"[맛나게, 잘, 먹었습니다~~~]"
3,"[맛있어요, 양도, 많아요]"
4,"[잘먹었어요, 또주문할께요]"


In [15]:
apply_regular_expression(chicken_review['리뷰'][2])

'맛나게 잘 먹었습니다'

In [16]:
tokenized_doc = chicken_review['리뷰'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc[:5])

0                 []
1      [양이, 많이, 줄어든]
2    [맛나게, 먹었습니다~~~]
3    [맛있어요, 양도, 많아요]
4    [잘먹었어요, 또주문할께요]
Name: 리뷰, dtype: object


## 3) TF-IDF 행렬 만들기

TF-IDF 실습에서 배운 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용한다.

이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행해보자.

In [17]:
# 역토큰화

detokenized_doc = []
for i in range(len(chicken_review['리뷰'])):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

chicken_review['리뷰'] = detokenized_doc

In [18]:
chicken_review[:5]

,리뷰
0,
1,양이 많이 줄어든
2,맛나게 먹었습니다~~~
3,맛있어요 양도 많아요
4,잘먹었어요 또주문할께요


In [19]:
# 상위 1000개의 단어를 보존
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(chicken_review['리뷰'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :', X.shape)

TF-IDF 행렬의 크기 : (99494, 1000)


## 4) 토픽 모델링

In [20]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)

In [21]:
lda_top = lda_model.fit_transform(X)

In [22]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00017452e-01 1.00022527e-01 1.00008455e-01 ... 1.00008001e-01
  1.12224661e-01 1.00024852e-01]
 [1.00004658e-01 1.00009442e-01 1.00005673e-01 ... 1.00009551e-01
  1.00027845e-01 1.00017347e-01]
 [1.00004210e-01 8.31432051e+01 1.00005515e-01 ... 1.00006776e-01
  5.90384586e+02 1.00009272e-01]
 ...
 [1.53718546e+02 1.00026990e-01 1.00018098e-01 ... 1.00019780e-01
  1.03792795e-01 1.00031692e-01]
 [1.00023415e-01 1.00017805e-01 3.86252772e+01 ... 1.00018816e-01
  1.00032766e-01 1.30545784e+02]
 [1.00011617e-01 1.00012630e-01 1.00005460e-01 ... 1.00014119e-01
  1.32044058e-01 1.00012848e-01]]
(10, 1000)


In [45]:
# 단어 집합. 1,000개의 단어가 저장됨
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names_out, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names_out[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

In [47]:
get_topics(lda_model.components_, terms)

Topic 1: [('양념도', 483.92), ('좋았습니다', 415.35), ('양도많고', 361.91), ('먹어요', 322.02), ('만족합니다', 296.87)]
Topic 2: [('먹었습니다', 1099.32), ('양념', 667.4), ('맛나요', 660.92), ('맛잇어요', 489.67), ('맛있음', 431.69)]
Topic 3: [('맛있습니다', 1727.77), ('후참잘', 590.38), ('맛나게', 567.4), ('맛있어용', 457.86), ('맛있는', 455.15)]
Topic 4: [('배달도', 1925.02), ('빠르고', 1811.46), ('배달', 1066.26), ('맛도', 993.84), ('양이', 773.86)]
Topic 5: [('최고', 593.13), ('먹었어요', 484.83), ('완전', 449.82), ('맛있어요', 383.47), ('감사합니다', 372.84)]
Topic 6: [('배달이', 806.84), ('ㅋㅋ', 460.53), ('믿고', 425.54), ('바삭하고', 419.31), ('먹는', 405.35)]
Topic 7: [('맛있어요', 4107.1), ('양도', 933.91), ('많고', 889.03), ('치킨은', 717.14), ('시켜', 366.27)]
Topic 8: [('후라이드', 1905.42), ('치킨', 1268.85), ('맛있었어요', 1134.01), ('잘먹었습니다', 869.81), ('맛있었습니다', 809.35)]
Topic 9: [('맛있네요', 1339.61), ('서비스', 509.79), ('양은', 495.58), ('서비스도', 443.75), ('맛잇게', 371.6)]
Topic 10: [('후라이드는', 990.04), ('잘먹었어요', 970.0), ('있습니다', 554.65), ('좋네요', 530.09), ('가성비', 498.72)]


In [48]:
tokenized_doc[:5]

0                 []
1      [양이, 많이, 줄어든]
2    [맛나게, 먹었습니다~~~]
3    [맛있어요, 양도, 많아요]
4    [잘먹었어요, 또주문할께요]
Name: 리뷰, dtype: object

In [49]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(0, 1), (1, 1), (2, 1)]


In [50]:
print(dictionary[66])

1닭했네요


In [51]:
len(dictionary)

114781

In [52]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.253*"배달도" + 0.174*"빠르고" + 0.075*"맛도" + 0.035*"배달"')
(1, '0.192*"치킨" + 0.054*"맛있는" + 0.047*"다른" + 0.029*"제일"')
(2, '0.056*"순살" + 0.049*"오늘은" + 0.031*"만족합니다" + 0.030*"시킬게요"')
(3, '0.196*"양도" + 0.092*"많고" + 0.050*"맛도" + 0.038*"서비스도"')
(4, '0.065*"^^" + 0.056*"그리고" + 0.053*"시켜봤는데" + 0.029*"굿굿"')
(5, '0.121*"양이" + 0.091*"맛은" + 0.044*"생각보다" + 0.043*"맛나게"')
(6, '0.055*"맛있어요~" + 0.044*"주문했는데" + 0.036*"좋았습니다" + 0.024*"시켜먹을게요"')
(7, '0.090*"치킨은" + 0.079*"많이" + 0.071*"후라이드는" + 0.028*"않고"')
(8, '0.053*"서비스로" + 0.052*"주신" + 0.034*"있어요" + 0.028*"치즈볼도"')
(9, '0.104*"배달" + 0.045*"시켜" + 0.044*"가성비" + 0.032*"혼자"')
(10, '0.114*"맛있습니다" + 0.044*"양은" + 0.043*"완전" + 0.042*"좋았어요"')
(11, '0.073*"후라이드가" + 0.059*"...." + 0.033*"소스가" + 0.028*"양념치킨"')
(12, '0.057*"양념이" + 0.047*"맛나요" + 0.047*"ㅋㅋ" + 0.047*"최고"')
(13, '0.229*"맛있네요" + 0.056*"먹었습니다~" + 0.033*"ㅜㅜ" + 0.028*"빨라서"')
(14, '0.170*"후라이드" + 0.065*"양념" + 0.043*"먹었는데" + 0.042*"좋네요"')
(15, '0.062*"배달이" + 0.053*"치킨이" + 0.049*"잘먹었어요" + 0.049*"배달은"')
(16, '0.

In [53]:
print(ldamodel.print_topics())

[(0, '0.253*"배달도" + 0.174*"빠르고" + 0.075*"맛도" + 0.035*"배달" + 0.023*"잘먹었습니다~" + 0.015*"간장" + 0.015*"역시나" + 0.012*"맛나네요" + 0.011*"배송도" + 0.010*"치킨도"'), (1, '0.192*"치킨" + 0.054*"맛있는" + 0.047*"다른" + 0.029*"제일" + 0.025*"앞으로" + 0.020*"입니다" + 0.018*"이제" + 0.017*"추천합니다" + 0.017*"여긴" + 0.016*"여기가"'), (2, '0.056*"순살" + 0.049*"오늘은" + 0.031*"만족합니다" + 0.030*"시킬게요" + 0.028*"먹었네요" + 0.026*"치킨을" + 0.023*"치즈볼" + 0.021*"그런데" + 0.021*"합니다" + 0.018*"같습니다"'), (3, '0.196*"양도" + 0.092*"많고" + 0.050*"맛도" + 0.038*"서비스도" + 0.038*"좋고" + 0.035*"여기" + 0.016*"해서" + 0.014*"좋아해서" + 0.014*"주문할게요" + 0.014*"먹는데"'), (4, '0.065*"^^" + 0.056*"그리고" + 0.053*"시켜봤는데" + 0.029*"굿굿" + 0.023*"먹으면" + 0.023*"깔끔하고" + 0.018*"시켜서" + 0.017*"넘나" + 0.015*"매콤하고" + 0.015*"있네요"'), (5, '0.121*"양이" + 0.091*"맛은" + 0.044*"생각보다" + 0.043*"맛나게" + 0.034*"사장님" + 0.023*"많아요" + 0.022*"다음에도" + 0.021*"모두" + 0.019*"배달이" + 0.017*"1시간"'), (6, '0.055*"맛있어요~" + 0.044*"주문했는데" + 0.036*"좋았습니다" + 0.024*"시켜먹을게요" + 0.022*"바삭바삭" + 0.019*"바로" + 0.017*"순살은" + 0.017*"맛과"

In [77]:
!pip install pyLDAvis

## LDA 시각화 하기

In [70]:
# !pip install pyLDAvis==2.1.2


In [83]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [56]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (4, 0.05), (5, 0.05), (6, 0.05), (7, 0.05), (8, 0.05), (9, 0.05), (10, 0.05), (11, 0.05), (12, 0.05), (13, 0.05), (14, 0.05), (15, 0.05), (16, 0.05), (17, 0.05), (18, 0.05), (19, 0.05)]
1 번째 문서의 topic 비율은 [(0, 0.012564488), (1, 0.012564488), (2, 0.012564488), (3, 0.012564488), (4, 0.2586989), (5, 0.26385278), (6, 0.012564488), (7, 0.26385203), (8, 0.012564488), (9, 0.012564488), (10, 0.012564488), (11, 0.012564488), (12, 0.012564488), (13, 0.012564488), (14, 0.012564488), (15, 0.012564488), (16, 0.012564488), (17, 0.012564488), (18, 0.012564488), (19, 0.012564488)]
2 번째 문서의 topic 비율은 [(0, 0.016667815), (1, 0.016667815), (2, 0.016667815), (3, 0.016667815), (4, 0.016667815), (5, 0.3500203), (6, 0.016667815), (7, 0.016667815), (8, 0.016667815), (9, 0.016667815), (10, 0.016667815), (11, 0.016667815), (12, 0.016667815), (13, 0.016667815), (14, 0.016667815), (15, 0.016667815), (16, 0.016667815), (17, 0.34995905), (18, 0.01666781

In [57]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [58]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0.0,0.0500,"[(0, 0.05), (1, 0.05), (2, 0.05), (3, 0.05), (..."
1,1,5.0,0.2639,"[(0, 0.012564439), (1, 0.012564439), (2, 0.012..."
2,2,5.0,0.3500,"[(0, 0.016667815), (1, 0.016667815), (2, 0.016..."
3,3,17.0,0.2625,"[(0, 0.012500113), (1, 0.012500113), (2, 0.012..."
4,4,15.0,0.5250,"[(0, 0.025000075), (1, 0.025000075), (2, 0.025..."
5,5,6.0,0.2277,"[(5, 0.11667774), (6, 0.22767133), (8, 0.11668..."
6,6,17.0,0.2100,"[(0, 0.0100009255), (1, 0.0100009255), (2, 0.0..."
7,7,15.0,0.3501,"[(0, 0.016673015), (1, 0.016673015), (2, 0.016..."
8,8,11.0,0.5036,"[(4, 0.07499597), (8, 0.07500627), (10, 0.0749..."
9,9,18.0,0.3500,"[(0, 0.01666699), (1, 0.01666699), (2, 0.01666..."
